# Keras实现Kaggle面部定点

这是我学习[这篇博文](https://elix-tech.github.io/ja/2016/06/02/kaggle-facial-keypoints-ja.html)时候的代码复现, 感谢原作者[Daniel Nouri]((http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/)基于[Lasagne](https://github.com/benanne/Lasagne)（[Theano](http://deeplearning.net/software/theano/)的高级框架）的实现，也感谢[真也雪](https://twitter.com/tuntuku_sy)将其迁移到Keras框架里。

## 数据集

[Kaggle数据集点此下载](https://www.kaggle.com/c/facial-keypoints-detection/data)

dataset  | num   | resolution | color     | keypoints
---      | ---   | ---        | ---       | ---
Training | 7049  | 96x96      | grayscale | 15

## 模型一：单隐层网络

### 载入数据集

图像在csv文件“Image”栏以空格分隔像素点，其余栏为定点数据。测试集仅有图像数据，无定点数据

In [ ]:
# 
import utils, importlib
importlib.reload(utils)

trainlib = 'training.csv'
testlib  = 'test.csv'

(X, y) = utils.loadset(trainlib)

print()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

In [ ]:
# 1-hidden-layer NN
import utils, importlib
importlib.reload(utils)

model = utils.modellib(name='single')
model.summary()

hist = model.fit(X, y, epochs=100, validation_split=0.2)

解释

1. dropna()之后只有2140样本，但是96x96=9216 input dimension 会远大于样本数量，容易导致过拟合。
1. 将X,y均归一化有利于防止算法异常（y放到[-1，1]可以用softmax?）
1. np.stack, np.hstack, np.vstack [用法和理解见这篇微博](http://blog.csdn.net/csdn15698845876/article/details/73380803)，scikit-learn的shuffle函数打乱后X,y的对应关系不会变（坐标还是对应原来图像）

In [ ]:
# 1-hidden-layer NN

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential()                # 建立顺序网络（简单的网络结构）
model.add(Dense(100,                # 中间全连接层，100个神经元
                input_dim=9216))    # 单个样本X的Shape，输入图像的像素数
model.add(Activation('relu'))       # 激活函数用Relu
model.add(Dense(30))                # 输出全连接层，30单元，单个y的Shape，15个点30个坐标

sgd = SGD(lr=0.01,                  # 自定义优化器
          momentum=0.9, 
          nesterov=True)
model.compile(loss='mean_squared_error', # 损失函数
              optimizer=sgd)             # 自定义的优化器
model.summary()                          # 显示网络结构
hist = model.fit(X, y, 
                 epochs=100,             # 训练轮数
                 validation_split=0.2)   # 20%的训练样本用于验证

### 损失函数可视化

应为用tf做后端，所以可以用tensorboard来进行可视化，此处用matplotlib

Keras的model.fit()对象存放有history，按标签读取绘图即可

In [ ]:
# Visualise
% matplotlib inline
import utils, importlib
importlib.reload(utils)

utils.histplot(hist, save='img/single-layer-test.png', show=True)

### 预测结果

In [ ]:
# prediction
import utils, importlib
importlib.reload(utils)

trainlib = 'training.csv'
testlib  = 'test.csv'

xtest, _ = utils.loadsets(testlib, test=True)
ypred    = model.predict(xtest)

In [ ]:
# show
import utils, importlib
importlib.reload(utils)

utils.predplot(xtest, ypred, save='img/pred-single.png', show=True)

### 保存、加载模型

In [ ]:
# save model
import utils, importlib
importlib.reload(utils)

# utils.savemodel(model, name='model/single_hidden_layer')

# load model
frame   = 'model/single_hidden_layer_architecture.json'
weights = 'model/single_hidden_layer_weights.h5'

# model = utils.loadmodel(frame, weights, toprint=True)

## 模型二 卷积神经网络

### 简介

组成：卷积层，汇合层，全连接层

1. 卷积层学习结果：每个卷积核的权重
![Conv feature][in1]

2. 汇合层：通过平均值汇合或最大值汇合降低上一层尺寸、防止过度学习，以减少参数和计算复杂度
![MaxPooling][in2]

3. 全连接层：即模型一中的隐层，与上一层所有神经元连接，上一层输出结果必须是一维数组

[in1]: img/intro-conv.gif
[in2]: img/intro-maxpool.jpg

### 模型结构

In [ ]:
# convert input-dim to (96,96,1)
import utils, importlib
importlib.reload(utils)

trainlib = 'training.csv'
testlib  = 'test.csv'

(X, y) = utils.loadset2d(trainlib)

print()
print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(X.shape, X.min(), X.max()))
print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(y.shape, y.min(), y.max()))

In [ ]:
# Model 2: CNN
import utils, importlib
importlib.reload(utils)

modelCNN = utils.modellib(name='CNN', dim=(96, 96, 1))
modelCNN.summary()

histCNN = modelCNN.fit(X, y, epochs=100, validation_split=0.2)

### 损失可视化

In [ ]:
# Visualise
% matplotlib inline
import utils, importlib
importlib.reload(utils)

utils.histplot(histCNN, save='img/CNN-test.png', show=True)

### 预测结果

In [ ]:
# prediction
import utils, importlib
importlib.reload(utils)

trainlib = 'training.csv'
testlib  = 'test.csv'

xtest, _ = utils.loadset2d(testlib, test=True)
ypred = modelCNN.predict(xtest)

In [ ]:
# show
import utils, importlib
importlib.reload(utils)

utils.predplot(xtest, ypred, save='img/pred-CNN.png', show=True)

### 保存载入

In [ ]:
# save model
import utils, importlib
importlib.reload(utils)

# utils.savemodel(modelCNN, name='model/CNN')

# load model
frame   = 'model/CNN_architecture.json'
weights = 'model/CNN_weights.h5'

# modelCNN = utils.loadmodel(frame, weights, toprint=True)

## 模型对比

In [ ]:
# generate preds
import utils, importlib
importlib.reload(utils)

trainlib = 'training.csv'
testlib  = 'test.csv'

xtest,    _= utils.loadset(testlib, test=True)
xtest2d, _ = utils.loadset2d(testlib, test=True)
ypred      = model.predict(xtest)
ypredCNN   = modelCNN.predict(xtest2d)

In [ ]:
# Compare loss
% matplotlib inline
import utils, importlib
importlib.reload(utils)

utils.histplotdiff(hist, histCNN, save='img/hist-compare.png', show=True)

In [ ]:
# Compare prediction
import utils, importlib
importlib.reload(utils)

utils.predplotdiff(xtest, ypred, ypredCNN, save='img/pred-compare.png', show=True)